In [13]:
import wandb
import pytorch_lightning as pl
import torch
from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, Resize, ToTensor

In [16]:
img_size = (512, 512)

def target_transform(target):
    size = float(target['annotation']['size']['width']), float(target['annotation']['size']['height']) 
    obj = [float(target['annotation']['object'][0]['bndbox'][coord]) for coord in ['xmin', 'ymin', 'xmax', 'ymax'] 
    label = 

dataset = VOCDetection(
    root='data',
    download=True,
    year='2007',
    image_set='val',
    transform=Compose([Resize(img_size), ToTensor()]),
    target_transform=
)

Using downloaded and verified file: data/VOCtrainval_06-Nov-2007.tar


In [5]:
wandb.init(project='test')

In [23]:
dataset[0][1]['annotation']['size']

{'width': '500', 'height': '375', 'depth': '3'}

In [18]:
from torch.utils.data import DataLoader

In [19]:
dataloader = DataLoader(dataset)

In [20]:
next(iter(dataloader))

[tensor([[[[0.0431, 0.0510, 0.0392,  ..., 0.6431, 0.6392, 0.6353],
           [0.0275, 0.0392, 0.0431,  ..., 0.6353, 0.6353, 0.6314],
           [0.0706, 0.0549, 0.0510,  ..., 0.6353, 0.6353, 0.6353],
           ...,
           [0.1412, 0.1333, 0.1294,  ..., 0.2471, 0.2588, 0.2549],
           [0.1647, 0.1765, 0.2000,  ..., 0.2549, 0.2706, 0.2706],
           [0.1922, 0.2392, 0.2824,  ..., 0.2510, 0.2667, 0.2706]],
 
          [[0.0353, 0.0510, 0.0392,  ..., 0.7333, 0.7294, 0.7255],
           [0.0235, 0.0392, 0.0431,  ..., 0.7373, 0.7333, 0.7333],
           [0.0745, 0.0588, 0.0510,  ..., 0.7412, 0.7412, 0.7412],
           ...,
           [0.0353, 0.0392, 0.0353,  ..., 0.3020, 0.3137, 0.3137],
           [0.0431, 0.0471, 0.0588,  ..., 0.3137, 0.3294, 0.3294],
           [0.0667, 0.0824, 0.0980,  ..., 0.3098, 0.3255, 0.3294]],
 
          [[0.0392, 0.0510, 0.0392,  ..., 0.7569, 0.7529, 0.7490],
           [0.0314, 0.0392, 0.0431,  ..., 0.7608, 0.7608, 0.7569],
           [0.0824, 0.06

Object detection tutorial

In [7]:
wandb.log({'loss': 0.4})

Distillation Tutorial